In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")
# define experiment
#exp_automl = Experiment(workspace=ws, name="automl")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-189948
Azure region: southcentralus
Subscription id: d7f39349-a66b-446e-aba6-0053c2cf1c11
Resource group: aml-quickstarts-189948


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "myComputeName"

try:
    compute_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    cluster_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, cluster_name, cluster_config)

compute_cluster.wait_for_completion(show_output=True)

# get a detailed status for the current cluster.
print(compute_cluster.get_status().serialize())

Found existing compute target
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2022-03-25T07:11:06.244000+00:00', 'errors': None, 'creationTime': '2022-03-25T06:54:02.830959+00:00', 'modifiedTime': '2022-03-25T06:54:06.538104+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS3_V2'}


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os, logging
logging.basicConfig(level=logging.INFO)

# Create a directory that will contain all the necessary code.
#project_folder = 'Final_Project'
#os.makedirs(project_folder, exist_ok=True)

# parameter sampler
param_sampling = RandomParameterSampling( {
        '--C': choice(0.1, 1, 10, 100, 1000), 
        #'--max_iter': uniform(1, 500)
        '--max_iter': choice(1, 50, 100, 200, 500)
    })

# End poorly performing runs with an early termination policy (improves computational efficiency).
early_termination_policy = BanditPolicy(slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of training job
src = ScriptRunConfig(source_directory='./',
                      script='train.py',
                      #arguments=['--C', '--max_iter'],
                      compute_target=compute_cluster,
                      environment=sklearn_env)

# Create a HyperDriveConfig.
# The name of the primary metric needs to exactly match the name of the metric logged by the training script.
hyperdrive_config = HyperDriveConfig(run_config=src,
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=4,
                             max_concurrent_runs=4)

In [4]:
# Submit the experiment (start the HyperDrive run).
hyperdrive_run = exp.submit(hyperdrive_config)

In [5]:
# Visualize all hyperparameter tuning runs.
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_33acce7c-6006-42f2-b2d3-fa29fdd585ca
Web View: https://ml.azure.com/runs/HD_33acce7c-6006-42f2-b2d3-fa29fdd585ca?wsid=/subscriptions/d7f39349-a66b-446e-aba6-0053c2cf1c11/resourcegroups/aml-quickstarts-189948/workspaces/quick-starts-ws-189948&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-03-25T07:18:50.427067][API][INFO]Experiment created<END>\n""<START>[2022-03-25T07:18:51.233878][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-03-25T07:18:53.003535][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_33acce7c-6006-42f2-b2d3-fa29fdd585ca
Web View: https://ml.azure.com/runs/HD_33acce7c-6006-42f2-b2d3-fa29fdd585ca?wsid=/subscriptions/d7f39349-a66b-446e-aba6-0053c2cf1c11/resourcegroups/aml-quickstarts-189948/workspaces/quick-starts-ws-189948&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_33acce7c-6006-42f2-b2d3-fa29fdd585ca',
 'target': 'myComputeName',
 'status': 'Completed',
 'startTimeUtc': '2022-03-25T07:18:50.140815Z',
 'endTimeUtc': '2022-03-25T07:23:26.797972Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '8a82f03a-4591-4dca-be26-bd1fa08a49cb',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1068-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.38.0',
  'space_size': '25',
  'score': '0.908649468892261',
  'best_child_run_id': 'HD_33acce7c-6006-42f2-b2d3-fa29fdd585ca_2',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg189948.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_33acce7c-6006-42f2-b2d3-fa29fdd585ca/azureml-

KeyError: 'log_files'

In [6]:
assert(hyperdrive_run.get_status() == "Completed")

In [7]:
import joblib

# Best performing configuration and hyperparameter values.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n learning rate:',parameter_values[3])
#print('\n keep probability:',parameter_values[5])
#print('\n batch size:',parameter_values[7])

# List the model files uploaded during the run.
# print('\n Model files:', best_run.get_file_names())

Best Run Id:  HD_33acce7c-6006-42f2-b2d3-fa29fdd585ca_2

 Accuracy: 0.908649468892261

 learning rate: 100


In [8]:
# Save the model from that run.
model_hyperdrive = best_run.register_model(model_name='model_hyperdrive', model_path='outputs/model.joblib')

In [9]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset
data_location = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=data_location, validate=True, include_path=False, infer_column_types=True, set_column_types=None, separator=',', header=True, partition_format=None, support_multi_line=False, empty_as_string=False)

In [10]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# clean data.
x, y = clean_data(ds)

# y is a pandas Series. Convert to dataframe.
y_df = pd.DataFrame(y,columns=['y'])

# Combine both dataframes : x and y_df.
combined_data = pd.concat([x,y_df],axis=1)

# Split data into train and test sets.
train_data, test_data = train_test_split(combined_data, test_size = 0.2, random_state = 42)

# Convert training data to csv and save in training folder.
train_data.to_csv('training/automl_training_data.csv')

# get datastore
datastore_automl = ws.get_default_datastore()

# upload data to datastore
datastore_automl.upload(src_dir='training/',target_path='data/')

# convert data to tabular format
automl_train_dataset = TabularDatasetFactory.from_delimited_files(path=[(datastore_automl,('data/automl_training_data.csv'))])

INFO:matplotlib.font_manager:generated new fontManager
INFO:azureml.data.datastore_client:<azureml.core.authentication.InteractiveLoginAuthentication object at 0x7f4098a6c588>
INFO:azureml.data.azure_storage_datastore:Called AzureBlobDatastore.upload
INFO:azureml.data.azure_storage_datastore:Uploading an estimated of 1 files
INFO:azureml.data.azure_storage_datastore.task_upload_data/automl_training_data.csv:Uploading training/automl_training_data.csv
INFO:azureml.data.azure_storage_datastore.task_upload_data/automl_training_data.csv:Uploaded training/automl_training_data.csv, 1 files out of an estimated total of 1
INFO:azureml.data.azure_storage_datastore:Uploaded 1 files
INFO:azureml.data.azure_storage_datastore:Finished AzureBlobDatastore.upload with count=1.


Uploading an estimated of 1 files


In [14]:
from azureml.train.automl import AutoMLConfig

automl_settings = {
       "n_cross_validations": 3,
       "primary_metric": 'accuracy',
       "experiment_timeout_minutes":30,
       "verbosity": logging.INFO,
   }

# Set parameters for AutoMLConfig
automl_config = AutoMLConfig(task = 'classification',
                             compute_target = compute_cluster,
                             training_data = automl_train_dataset,
                             label_column_name='y',
                             **automl_settings
                             )

In [15]:
#  Submit automl run (show Validation errors and current status).
automl_run = exp.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on myComputeName with default configuration
Running on remote compute: myComputeName


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_c338fdcc-54cd-4b47-8c90-68720e2b3ecc,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Nu

In [16]:
from azureml.widgets import RunDetails

RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_c338fdcc-54cd-4b47-8c90-68720e2b3ecc,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|2951                          |1                               |26360                                 |
+------------------------------+--------------------------------+----

{'runId': 'AutoML_c338fdcc-54cd-4b47-8c90-68720e2b3ecc',
 'target': 'myComputeName',
 'status': 'Completed',
 'startTimeUtc': '2022-03-25T07:34:22.359903Z',
 'endTimeUtc': '2022-03-25T08:02:35.51264Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'myComputeName',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"d7f39349-a66b-446e-aba6-0053c2cf1c11","resource_group":"aml-quickstarts-189948","workspace_name":"quick-starts-ws-189948","region":"southcentralus","compute_target":"myComputeNa

Current provisioning state of AmlCompute is "Deleting"



In [17]:
# Retrieve the best Run object
#automl_best_run = automl_run.get_best_child()

# Retrieve the best run and the fitted model.
automl_best_run, fitted_model = automl_run.get_output()

INFO:pytorch_transformers.modeling_bert:Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
INFO:pytorch_transformers.modeling_xlnet:Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [18]:
# Save best automl model.
automl_best_run.register_model(model_name = "automl_model.pkl", model_path = './outputs/')
print(fitted_model._final_estimator)

PreFittedSoftVotingClassifier(
    estimators=[('1', Pipeline(
        memory=None,
        steps=[('maxabsscaler', MaxAbsScaler(
            copy=True
        )), ('xgboostclassifier', XGBoostClassifier(
            random_state=0,
            n_jobs=1,
            problem_info=ProblemInfo(
                gpu_training_param_dict={'processing_unit_type': 'cpu'}
            ),
            tree_method='auto'
        ))],
        verbose=False
    )), ('0', Pipeline(
        memory=None,
        steps=[('maxabsscaler', MaxAbsScaler(
            copy=True
        )), ('lightgbmclassifier', LightGBMClassifier(
            min_data_in_leaf=20,
            random_state=None,
            n_jobs=1,
            problem_info=ProblemInfo(
                gpu_training_param_dict={'processing_unit_type': 'cpu'}
            )
        ))],
        verbose=False
    )), ('14', Pipeline(
        memory=None,
        steps=[('standardscalerwrapper', StandardScalerWrapper(
            copy=True,
        

In [19]:
# delete compute cluster
compute_cluster.delete()